In [1]:
# import relevant libraries
import os
import numpy as np
import pandas as pd

In [75]:
graph_df = pd.DataFrame([], columns=['DCT_{i}'.format(i=i) for i in range(1, 16)])

for QF1 in [60, 65, 70]:
    for QF2 in [90]:
        row = []
        df = pd.read_csv(os.path.join('Result', 'QF1{q1}_QF2{q2}.csv'.format(q1=QF1, q2=QF2)))
        for i in range(1, 16):
            row.append(df[f'DCT_{i}'].sum() / df.shape[0])
        
        graph_df = pd.concat([graph_df, pd.DataFrame([row], columns=graph_df.columns)], ignore_index=True)

# graph_df['mean'] = graph_df.mean(axis=1)
graph_df.head()

,DCT_1,DCT_2,DCT_3,DCT_4,DCT_5,DCT_6,DCT_7,DCT_8,DCT_9,DCT_10,DCT_11,DCT_12,DCT_13,DCT_14,DCT_15
0,0.844,0.804,0.860,0.776,0.844,0.820,0.684,0.776,0.780,0.716,0.652,0.616,0.636,0.584,0.308
1,0.796,0.860,0.840,0.772,0.888,0.812,0.732,0.772,0.776,0.768,0.556,0.664,0.716,0.520,0.500
2,0.804,0.724,0.784,0.836,0.828,0.864,0.736,0.792,0.796,0.800,0.532,0.752,0.756,0.604,0.352


In [ ]:
# Get the average of each DCT coefficients 
final_row = []

for i in range(1, 16):
    final_row.append(graph_df['DCT_{i}'.format(i=i)].mean())

graph_df = pd.DataFrame([final_row], columns=graph_df.columns)
ax = graph_df.T.plot(legend=False, grid=True, figsize=(10, 5), use_index=True)

graph_df.head()

In [37]:
def insert_reference_row(df):
    res_df = pd.DataFrame(columns=df.columns)
    ref = df.iloc[[0]]

    for idx, row in df.iterrows():
        if idx == 0:
            continue
        res_df = pd.concat([res_df, ref, pd.DataFrame([row], columns=res_df.columns)], ignore_index=True)
    
    return res_df

In [136]:
df = pd.read_csv(os.path.join('Result', '6_10_2022', 'test_QF1100_QF280.csv'))
res_df = insert_reference_row(df)

def row_style(row):
    if row['File'] != 'Reference table':
        return pd.Series('background-color: #9fcbee', row.index) #light blue
    else:
        return pd.Series('background-color: #6497bf', row.index) #dark blue

res_df.style.apply(row_style, axis=1)

,File,DCT_1,DCT_2,DCT_3,DCT_4,DCT_5,DCT_6,DCT_7,DCT_8,DCT_9,DCT_10,DCT_11,DCT_12,DCT_13,DCT_14,DCT_15
0,Reference table,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,r000da54ft.TIF,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [123]:
def evaluate_result_for_csv(csv_file, append_df, is_accuracy):
    df = pd.read_csv(csv_file)
    local_df = pd.DataFrame([], columns=[['csv'] + ['DCT_{i}'.format(i=i) for i in range(1, 16)]])
    ref = df.iloc[0].to_numpy()
    row_to_be_append = [csv_file]

    for idx, row in df.iterrows():
        if idx == 0:
            continue

        accuracy_row = [csv_file]
        np_row = row.to_numpy()

        for i in range(1, 16):
            if is_accuracy:
                accuracy_row.append(
                    1 if np_row[i] == ref[i] else 0
                )
            else:
                # do a error estimation
                accuracy_row.append(
                    round(abs(ref[i] - np_row[i]) / ref[i], 4)
                )
        
        local_df = pd.concat([local_df, pd.DataFrame([accuracy_row], columns=local_df.columns)], ignore_index=True)
    
    for i in range(1, 16):
        row_to_be_append.append(local_df[f'DCT_{i}'].sum().iloc[0] / local_df.shape[0])

    append_df = pd.concat([append_df, pd.DataFrame([row_to_be_append], columns=append_df.columns)], ignore_index=True)

    return append_df

def evalaute_results(dir, is_accuracy):
    df = pd.DataFrame([], columns=[['csv'] + ['DCT_{i}'.format(i=i) for i in range(1, 16)]])
    
    for QF1 in [60, 65, 70, 75, 80, 85, 90, 95]:
        for QF2 in [80, 90]:
            csv_file = os.path.join(dir, 'test_QF1{q1}_QF2{q2}.csv'.format(q1=QF1, q2=QF2))
            
            df = evaluate_result_for_csv(csv_file=csv_file, append_df=df, is_accuracy=is_accuracy)
    
    return df

In [131]:
accuracy = evalaute_results(os.path.join('Result', '6_10_2022'), is_accuracy=True)
# merge columns
print(accuracy.mean(axis=0, numeric_only=True))

# merge rows
s1 = accuracy['csv']
s1.name = 'csv'

s2 = accuracy.mean(axis=1, numeric_only=True)
s2.name = 'accuracy'

pd.concat([s1, s2], axis=1)

DCT_1     0.60625
DCT_2     0.57500
DCT_3     0.61250
DCT_4     0.55625
DCT_5     0.63125
DCT_6     0.58750
DCT_7     0.55000
DCT_8     0.58125
DCT_9     0.60000
DCT_10    0.51250
DCT_11    0.33750
DCT_12    0.55000
DCT_13    0.50625
DCT_14    0.51250
DCT_15    0.39375
dtype: float64


,"(csv,)",accuracy
0,Result\6_10_2022\test_QF160_QF280.csv,0.553333
1,Result\6_10_2022\test_QF160_QF290.csv,0.720000
2,Result\6_10_2022\test_QF165_QF280.csv,0.466667
3,Result\6_10_2022\test_QF165_QF290.csv,0.700000
4,Result\6_10_2022\test_QF170_QF280.csv,0.620000
5,Result\6_10_2022\test_QF170_QF290.csv,0.726667
6,Result\6_10_2022\test_QF175_QF280.csv,0.453333
7,Result\6_10_2022\test_QF175_QF290.csv,0.793333
8,Result\6_10_2022\test_QF180_QF280.csv,0.060000
9,Result\6_10_2022\test_QF180_QF290.csv,0.793333


In [130]:
error = evalaute_results(os.path.join('Result', '6_10_2022'), is_accuracy=False)
# merge columns
print(error.mean(axis=0, numeric_only=True))

# merge rows
s3 = error['csv']
s3.name = 'csv'

s4 = error.mean(axis=1, numeric_only=True)
s4.name = 'error'

pd.concat([s3, s4], axis=1)

DCT_1     0.179634
DCT_2     0.279899
DCT_3     0.219905
DCT_4     0.199451
DCT_5     0.229211
DCT_6     0.253312
DCT_7     0.181326
DCT_8     0.234496
DCT_9     0.194722
DCT_10    0.233936
DCT_11    0.253970
DCT_12    0.173499
DCT_13    0.215265
DCT_14    0.192386
DCT_15    0.296057
dtype: float64


,"(csv,)",error
0,Result\6_10_2022\test_QF160_QF280.csv,0.126981
1,Result\6_10_2022\test_QF160_QF290.csv,0.106223
2,Result\6_10_2022\test_QF165_QF280.csv,0.185191
3,Result\6_10_2022\test_QF165_QF290.csv,0.102405
4,Result\6_10_2022\test_QF170_QF280.csv,0.173877
5,Result\6_10_2022\test_QF170_QF290.csv,0.100250
6,Result\6_10_2022\test_QF175_QF280.csv,0.270482
7,Result\6_10_2022\test_QF175_QF290.csv,0.056401
8,Result\6_10_2022\test_QF180_QF280.csv,0.449793
9,Result\6_10_2022\test_QF180_QF290.csv,0.062763
